In [1]:
import pyspark
import numpy as np
import pandas as pd
import os
import time
import datetime
from os import walk
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import *
from pyspark.sql.functions import date_format
from datetime import date, timedelta
from tqdm.notebook import tqdm
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [2]:
def Sort_Tuple(tup): 
    tup.sort(key = lambda x: x[1]) 
    return tup
#path_list = Sort_Tuple(path_list)

import re
def my_split(s):
    return list(filter(None, re.split(r'(\d+)', s)))
#print(path_list)
#print(len(path_list))
def get_symbol(tic):
    li = list(filter(None, re.split(r'(\d+)', tic)))
    return li[0]

# Read data from database and convert it into pandas dataframe

In [3]:
############################################## INPUTS #####################################################
startDate = date(2023,1,18)
endDate = date(2023,1,18)
#startDate = endDate = date.today()

total_data_combined = 0
ignore_symbols = ['NIFTY', 'BANKNIFTY', 'FINNIFTY']
numberOfRowsRaw = 0
start_time = datetime.datetime.strptime('09:15:00', '%H:%M:%S').time()
end_time = datetime.datetime.strptime('15:30:00', '%H:%M:%S').time()
###########################################################################################################
print(startDate)

2023-01-18


In [4]:
print(datetime.datetime.now())

# Start spark session
spark = SparkSession.builder.config("spark.jars", "C:\\Users\\ADMIN\\Downloads\\postgresql-42.5.0.jar") \
        .master("local").appName("PySpark_Postgres_test").getOrCreate()
print(spark)

# Remove previous day files
new_data_path = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Log_Data\\"
log_data_path = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Temp_Data\\"

for file in next(os.walk(new_data_path))[2]:
    os.remove(new_data_path + file)    

for file in next(os.walk(log_data_path))[2]:
    os.remove(log_data_path + file)
    
misc_data_path = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Temp_Data\misc_data_afterstockwise\\"
log_misc_data_path = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Log_Data\misc_data_afterstockwise\\"

for file in next(os.walk(misc_data_path))[2]:
    os.remove(log_data_path + file)
    
for file in next(os.walk(log_misc_data_path))[2]:
    os.remove(log_data_path + file)

# Loop through all the dates to create symbolwise files
for n in tqdm(range(int((endDate - startDate).days)+1)):
    currentDate = startDate + timedelta(n)
        
    day = currentDate.strftime('%d')
    nummonth=currentDate.strftime("%m")
    year=currentDate.strftime('%Y')
    tablename="r"+str(day)+str(nummonth)+str(year)
    #print(tablename)

    st=time.time()

    # Check if file is available in the database
    conn = psycopg2.connect(database="RawDataBase",
                        user='postgres', password='swancap123',
                        host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432')
    cursor = conn.cursor()
    stmt = '''Select 1 from rawinfo where name=\'''' + tablename + '''\';'''
    cursor.execute(stmt)
    result = cursor.fetchone()
    
    if result:
        print(tablename)
    else:
        continue
    
    
    df = spark.read.format("jdbc").option("url", "jdbc:postgresql://swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com/RawDataBase").option("user","postgres").option("password","swancap123")\
        .option("driver", "org.postgresql.Driver").option("dbtable", tablename)\
        .option("user", "postgres").option("password", "swancap123").load()

    # GETTING ONLY TIME IN TIME COLUMN
    q = df.withColumn('Time',date_format('Time', 'HH:mm:ss'))

    # FILTERING BANK NIFTY DATA
    bndata = q.filter((~q.ticker.contains('NIFTY')))
    bndata = bndata.filter(bndata.ticker.endswith('E.NFO'))

    # CONVERTING PYSPARK DATAFRAME TO PANDAS DATAFRAME
    bndata=bndata.toPandas()

    data = bndata.copy()
    data.drop(data.filter(regex="Unname"),axis=1, inplace=True)
    
    data = data.rename(columns={'Ticker ' : 'Ticker',
                                'ticker' : 'Ticker',
                                'date' : 'Date',
                                'time' : 'Time',
                                'open' : 'Open',
                                'high' : 'High',
                                'low' : 'Low',
                                'close' : 'Close',
                                'volume' : 'Volume',
                                'open interest' : 'Open Interest'})
    
    data["Ticker"] = data["Ticker"].str.replace("HINDALC0","HINDALCO")
    data["Ticker"] = data["Ticker"].str.replace("IBN18","IBN_ET")
    data["Ticker"] = data["Ticker"].str.replace("TV-18","TV_ET")
    data["Ticker"] = data["Ticker"].str.replace("NETWORK18","NETWORK_ET")
    data["Ticker"] = data["Ticker"].str.replace("TV18BRDCST","TV_ETBRDCST")
    data["Ticker"] = data["Ticker"].str.replace("3IINFOTECH","TH_IINFOTECH")
    data["Ticker"] = data["Ticker"].str.replace("BAJAJ-AUTO","BAJAJ_AUTO")
    data["New_date"] = currentDate
    print(currentDate)
    print("Total Data Size "+ str(data.shape[0]))
    
    data["symbol"] = data["Ticker"].str.split("(\d+)").str[0]
    
    fut_data = data[data["Ticker"].str.endswith(("I","II","III", 'I.NFO', 'II.NFO', 'III.NFO'))] 
    data = data[~data["Ticker"].str.endswith(("I","II","III"))]
    misc_data = data[~data["Ticker"].str.endswith(('.NFO', 'CE', 'PE'))]
    data = data[data["Ticker"].str.endswith(("PE","CE","PE.NFO","CE.NFO"))]
    
    
    data = data[~data['symbol'].isin(ignore_symbols)]
    data['Time'] = pd.to_datetime(data['Time']).dt.time
    data = data[(data['Time']>=start_time) & (data['Time']<=end_time)]
    
    numberOfRowsRaw = data.shape[0]
    total_data_combined = total_data_combined + data.shape[0]
       
    gb = data.groupby(["symbol"])
    df_list = [gb.get_group(x) for x in gb.groups]
       
    for tup in tqdm(gb.groups):
        temp = gb.get_group(tup)
        
        temp.to_csv(new_data_path + str(tup)  + '.csv', mode='a', header=not os.path.exists(new_data_path + str(tup)  + '.csv'), index=False)
        temp.to_csv(log_data_path + str(tup)  + '.csv', mode='a', header=not os.path.exists(log_data_path + str(tup)  + '.csv'), index=False)   

    print("Processing Finished")  
        
    gb = misc_data.groupby(["symbol"])
    df_list = [gb.get_group(x) for x in gb.groups]
       
    for tup in tqdm(gb.groups):
        temp = gb.get_group(tup)       
        
        temp.to_csv(misc_data_path + str(tup) + '.csv', mode='a', header=not os.path.exists(misc_data_path + str(tup) + '.csv'), index=False)
        temp.to_csv(log_misc_data_path + str(tup)  + '.csv', mode='a', header=not os.path.exists(log_misc_data_path + str(tup)  + '.csv'), index=False)
     
    print("Data appending finished")
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print(current_time)
    
    print("Total Rows read from files till now " + str(total_data_combined))
    
    mypath = new_data_path
    filename = next(walk(mypath), (None, None, []))[2]  # [] if no file
    final = []
    for i in filename:
        temp = mypath +"/"+ i
        final.append(temp)
    sum1 = 0
    for i in final:
        df = pd.read_csv(i)
        sum1 += df.shape[0]
    print("Total Rows in csv files" + str(sum1))
print(datetime.datetime.now())

2023-01-19 09:24:47.549696


  0%|          | 0/1 [00:00<?, ?it/s]

r18012023
2023-01-18
Total Data Size 532583


  0%|          | 0/191 [00:00<?, ?it/s]

Processing Finished


0it [00:00, ?it/s]

Data appending finished
09:25:19
Total Rows read from files till now 531929
Total Rows in csv files531929
2023-01-19 09:25:21.342799


# Labelling

In [5]:
date1 = datetime.datetime.strptime('30-12-2019', '%d-%m-%Y')
new_data_path = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Temp_Data\\"

In [6]:
print(datetime.datetime.now())
mypath = new_data_path
filename = next(walk(mypath), (None, None, []))[2]  # [] if no file
file_path = {}
for i in filename:
    temp = mypath +"/"+ i
    file_path[i.replace(".csv","")] = temp

labeled_data_path = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Temp_Data\labeled_data\\"
log_labeled_data_path = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Log_Data\labeled_data\\"

for i in next(os.walk(labeled_data_path))[2]:
    os.remove(labeled_data_path + i)

for i in next(os.walk(log_labeled_data_path))[2]:
    os.remove(log_labeled_data_path + i)

for sym in tqdm(sorted(file_path)):
    print(sym)
    data = pd.read_csv(file_path[sym], chunksize = 100000)

    for df in data:
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        df = df.drop_duplicates()
        df = df.drop(columns = ["symbol"])
        
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
        df['Ticker'] = df['Ticker'].str.replace('.NFO', '')
        df["Symbol"] = sym
        df["Option_type"] = df["Ticker"].str[-2:]
        df["Temp"] = df["Ticker"].str.replace(sym,"")        
        df["Temp"] = df["Temp"].str[:-2]
        
        df['Exp_year'] = np.where(df['Date']>=date1, df["Temp"].str[5:7], df['Temp'].str[0:2])
        #df["Exp_year"] = df["Temp"].str[5:7] # change Exp_year
        df["Exp_month"] = df["Temp"].str[2:5]
        df['Strike'] = np.where(df['Date']>=date1, df["Temp"].str[7:], df['Temp'].str[5:])
        
        df = df.rename(columns={'Time' : 'Timestamp'})
        df['New_date_Time'] = pd.to_datetime(df['New_date'] + ' ' + df['Timestamp'])
        
        #df["Strike"] = df["Temp"].str[7:] # change Strike
        df.to_csv(labeled_data_path + sym + '.csv', mode='a', header=not os.path.exists(labeled_data_path + sym + '.csv'), index=False)    
        df.to_csv(log_labeled_data_path + sym + '.csv', mode='a', header=not os.path.exists(log_labeled_data_path + sym + '.csv'), index=False)
        
        del df
        
print(datetime.datetime.now())

2023-01-19 09:26:00.064960


  0%|          | 0/191 [00:00<?, ?it/s]

AARTIIND
ABB
ABBOTINDIA
ABCAPITAL
ABFRL
ACC
ADANIENT
ADANIPORTS
ALKEM
AMBUJACEM
APOLLOHOSP
APOLLOTYRE
ASHOKLEY
ASIANPAINT
ASTRAL
ATUL
AUBANK
AUROPHARMA
AXISBANK
BAJAJFINSV
BAJAJ_AUTO
BAJFINANCE
BALKRISIND
BALRAMCHIN
BANDHANBNK
BANKBARODA
BATAINDIA
BEL
BERGEPAINT
BHARATFORG
BHARTIARTL
BHEL
BIOCON
BOSCHLTD
BPCL
BRITANNIA
BSOFT
CANBK
CANFINHOME
CHAMBLFERT
CHOLAFIN
CIPLA
COALINDIA
COFORGE
COLPAL
CONCOR
COROMANDEL
CROMPTON
CUB
CUMMINSIND
DABUR
DALBHARAT
DEEPAKNTR
DELTACORP
DIVISLAB
DIXON
DLF
DRREDDY
EICHERMOT
ESCORTS
EXIDEIND
FEDERALBNK
FSL
GAIL
GLENMARK
GMRINFRA
GNFC
GODREJCP
GODREJPROP
GRANULES
GRASIM
GUJGASLTD
HAL
HAVELLS
HCLTECH
HDFC
HDFCAMC
HDFCBANK
HDFCLIFE
HEROMOTOCO
HINDALCO
HINDCOPPER
HINDPETRO
HINDUNILVR
HONAUT
IBULHSGFIN
ICICIBANK
ICICIGI
ICICIPRULI
IDEA
IDFC
IDFCFIRSTB
IEX
IGL
INDHOTEL
INDIACEM
INDIAMART
INDIGO
INDUSINDBK
INDUSTOWER
INFY
INTELLECT
IOC
IPCALAB
IRCTC
ITC
JINDALSTEL
JKCEMENT
JSWSTEEL
JUBLFOOD
KOTAKBANK
L&TFH
LALPATHLAB
LAURUSLABS
LICHSGFIN
LT
LTIM
LTTS
LUPIN
M&M
M&

# Continuous Contracts

In [7]:
def add(stri):
    obj = datetime.datetime.strptime(stri, "%b")
    month_number = obj.month
    return month_number

In [8]:
mypath = labeled_data_path
filename = next(walk(mypath), (None, None, []))[2]  # [] if no file
file_path = {}

for i in filename:
    temp = mypath +"/"+ i
    file_path[i.replace(".csv","")] = temp

exp_file_path = r"E:\sourav\Database\Codes\CsvFiles\MonthlyExpiry.csv"
exp_df = pd.read_csv(exp_file_path,parse_dates = ["curr_exp_date","curr_date"],dayfirst =True,usecols = ["curr_exp_date","curr_date"]).dropna()
exp_df.rename({'curr_date': 'New_date'}, axis=1, inplace=True)

print(exp_df.isnull().values.any())

folpath = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Temp_Data\cont_data\\"
folpath_log = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Log_Data\cont_data\\"
diff_list_dict = {}

False


In [9]:
print(datetime.datetime.now())

for file in next(os.walk(folpath))[2]:
    os.remove(folpath + file)

for file in next(os.walk(folpath_log))[2]:
    os.remove(folpath_log + file)

for sym in tqdm(sorted(file_path)):
    
    print(sym)
    df = pd.read_csv(file_path[sym],parse_dates = ["New_date"],dayfirst =True)

    if df.empty == True:
        continue
        
    if sym == 'S&P':
        continue
        
    df['exp_month_number'] = df.apply(lambda row : add(row["Exp_month"]), axis = 1)
    df["New_date"] = pd.to_datetime(df["New_date"], dayfirst=True)
    df["current_month_number"] = df['New_date'].dt.month
    df["difference"] = df['exp_month_number'].astype(int) - df["current_month_number"].astype(int)
    
    df1 = pd.merge(df, exp_df, on ='New_date', how ='left')
    

    df1.drop(df1.filter(regex="Unname"),axis=1, inplace=True)
    print(df1.shape[0])
    df1["current_exp_month_number"] = df1['curr_exp_date'].dt.month
    df1["Diff_months"] = df1["current_exp_month_number"] - df1["current_month_number"]
    df1["Diff_months"] = df1["Diff_months"].astype(int) 

    bdf = df1[df1["Diff_months"] == 0]
    adf = df1[(df1["Diff_months"] == 1) | (df1["Diff_months"] == -11)]
    
    if bdf.shape[0] + adf.shape[0] == df1.shape[0]:
        print("Sanity Check Success")
    else:
        print("Error1")
        break
    
    agb = adf.groupby(["difference"])
    unique_val_list_a = list(adf["difference"].unique())
    bgb = bdf.groupby(["difference"])
    unique_val_list_b = list(bdf["difference"].unique())

    for i in unique_val_list_b:
        temp_df = bgb.get_group(i)
        
        if i == 0:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)
            temp_df.to_csv(folpath_log + sym + '-I.csv', mode='a', header=not os.path.exists(folpath_log + sym + '-I.csv'), index=False) 
        
        elif i == 1 or i == -11:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)
            temp_df.to_csv(folpath_log + sym + '-II.csv', mode='a', header=not os.path.exists(folpath_log + sym + '-II.csv'), index=False)
        
        elif i == 2 or i == -10:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)
            temp_df.to_csv(folpath_log + sym + '-III.csv', mode='a', header=not os.path.exists(folpath_log + sym + '-III.csv'), index=False)

        else:
            temp_df.to_csv(folpath + sym + 'misc.csv', mode='a', header=not os.path.exists(folpath + sym + 'misc.csv'), index=False)
            temp_df.to_csv(folpath_log + sym + 'misc.csv', mode='a', header=not os.path.exists(folpath_log + sym + 'misc.csv'), index=False)
         
    for i in unique_val_list_a:
        temp_df = agb.get_group(i)
        
        if i == 1 or i == -11:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)
            temp_df.to_csv(folpath_log + sym + '-I.csv', mode='a', header=not os.path.exists(folpath_log + sym + '-I.csv'), index=False)
    
        elif i == 2 or i == -10:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)
            temp_df.to_csv(folpath_log + sym + '-II.csv', mode='a', header=not os.path.exists(folpath_log + sym + '-II.csv'), index=False)
                           
        elif i == 3 or i == -9:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)               
            temp_df.to_csv(folpath_log + sym + '-III.csv', mode='a', header=not os.path.exists(folpath_log + sym + '-III.csv'), index=False)               
                        
        else:
            temp_df.to_csv(folpath + sym +'misc.csv', mode='a', header=not os.path.exists(folpath + sym +'misc.csv'), index=False)
            temp_df.to_csv(folpath_log + sym + 'misc.csv', mode='a', header=not os.path.exists(folpath_log + sym + 'misc.csv'), index=False)               
            
        
    list_files = os.listdir(folpath)
    sum1 = 0
    for i in list_files:
        if i.startswith(sym):
            dff = pd.read_csv(folpath + i)
            sum1 += dff.shape[0]
    if sum1 == df1.shape[0]:
        print("Sanity check2 Success")
    else:
        print("Error2")
        break
    print(sum1)
    
print(datetime.datetime.now())

2023-01-19 09:27:52.268329


  0%|          | 0/191 [00:00<?, ?it/s]

AARTIIND
1080
Sanity Check Success
Sanity check2 Success
1080
ABB
815
Sanity Check Success
Sanity check2 Success
815
ABBOTINDIA
119
Sanity Check Success
Sanity check2 Success
119
ABCAPITAL
675
Sanity Check Success
Sanity check2 Success
675
ABFRL
1336
Sanity Check Success
Sanity check2 Success
1336
ACC
1839
Sanity Check Success
Sanity check2 Success
1839
ADANIENT
14939
Sanity Check Success
Sanity check2 Success
14939
ADANIPORTS
6591
Sanity Check Success
Sanity check2 Success
6591
ALKEM
128
Sanity Check Success
Sanity check2 Success
128
AMBUJACEM
3446
Sanity Check Success
Sanity check2 Success
3446
APOLLOHOSP
4052
Sanity Check Success
Sanity check2 Success
4052
APOLLOTYRE
1993
Sanity Check Success
Sanity check2 Success
1993
ASHOKLEY
2125
Sanity Check Success
Sanity check2 Success
2125
ASIANPAINT
8183
Sanity Check Success
Sanity check2 Success
8183
ASTRAL
1422
Sanity Check Success
Sanity check2 Success
1422
ATUL
172
Sanity Check Success
Sanity check2 Success
172
AUBANK
1772
Sanity Check S

Sanity check2 Success
2130
NTPC
2001
Sanity Check Success
Sanity check2 Success
2001
OBEROIRLTY
446
Sanity Check Success
Sanity check2 Success
446
OFSS
785
Sanity Check Success
Sanity check2 Success
785
ONGC
2913
Sanity Check Success
Sanity check2 Success
2913
PAGEIND
571
Sanity Check Success
Sanity check2 Success
571
PEL
1221
Sanity Check Success
Sanity check2 Success
1221
PERSISTENT
3194
Sanity Check Success
Sanity check2 Success
3194
PETRONET
586
Sanity Check Success
Sanity check2 Success
586
PFC
3448
Sanity Check Success
Sanity check2 Success
3448
PIDILITIND
2059
Sanity Check Success
Sanity check2 Success
2059
PIIND
241
Sanity Check Success
Sanity check2 Success
241
PNB
4140
Sanity Check Success
Sanity check2 Success
4140
POLYCAB
2384
Sanity Check Success
Sanity check2 Success
2384
POWERGRID
2347
Sanity Check Success
Sanity check2 Success
2347
PVR
3530
Sanity Check Success
Sanity check2 Success
3530
RAIN
1033
Sanity Check Success
Sanity check2 Success
1033
RAMCOCEM
217
Sanity Check

# Split Adjusted Data 

In [10]:
final_data_path = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Temp_Data\cont_data\\"
#output_path = r"E:\sourav\Database\OptionsDatabaseDailyUpdate\Updation_Temp_Data\split_adjusted_final\\"
output_path = r"D:\Sourav\Data\UpdatedTill23Dec2022\\"
symbols = next(os.walk(final_data_path))[2]
len(symbols)

426

In [11]:
for i in range(len(symbols)):
    symbols[i] = symbols[i].replace('-III.csv', '').replace('-II.csv', '').replace('-I.csv', '')    
symbols = sorted(list(set(symbols)))
print(len(symbols))

191


In [12]:
hyphens = ['-I', '-II', '-III']
numberOfRowsFinal = 0

for symbol in tqdm(symbols):
    print(symbol)  
    sym1 = re.sub('[^a-zA-Z0-9 \n\.]', '', symbol)
    df_final = pd.DataFrame()
    
    for i in hyphens:    
        print(i)
        file_path = final_data_path + '\\' + symbol + i + '.csv'
        
        if os.path.isfile(file_path):
            
            df = pd.read_csv(file_path, dayfirst=True)

            df['Date'] = pd.to_datetime(df['Date'].astype(str), dayfirst=True).dt.date
            df = df.drop(['Date'], axis=1)
            df = df.rename(columns={'New_date' : 'Date'})
            
            df['Adj_Open'] = df['Open']
            df['Adj_High'] = df['High']
            df['Adj_Low'] = df['Low']
            df['Adj_Close'] = df['Close']
            df['Adj_strike'] = df['Strike']
            df['Adj_volume'] = df['Volume']
            df['Adj_OI'] = df['Open Interest']

            df['Final_strike'] = df['Strike']
            df['month'] = pd.to_datetime(df['Date'], dayfirst=True).dt.month    
            
            df = df.sort_values(by=['Date', 'Timestamp', 'Option_type', 'Strike'])
            df = df[['New_date_Time', 'Ticker', 'Date', 'Open', 'High', 'Low', 'Close',
                     'Volume', 'Open Interest', 'Symbol', 'Option_type', 'Temp', 'Exp_year',
                     'Exp_month', 'Strike', 'exp_month_number', 'current_month_number',
                     'difference', 'curr_exp_date', 'current_exp_month_number',
                     'Diff_months', 'Timestamp', 'Adj_Open', 'Adj_High', 'Adj_Low',
                     'Adj_Close', 'Adj_volume', 'Adj_OI', 'Adj_strike', 'Final_strike',
                     'month']]
            numberOfRowsFinal += df.shape[0]
            df.to_csv(output_path + symbol + i + '.csv', mode='a', header=not os.path.exists(output_path + symbol + i + '.csv'), index=False)

  0%|          | 0/191 [00:00<?, ?it/s]

AARTIIND
-I
-II
-III
ABB
-I
-II
-III
ABBOTINDIA
-I
-II
-III
ABCAPITAL
-I
-II
-III
ABFRL
-I
-II
-III
ACC
-I
-II
-III
ADANIENT
-I
-II
-III
ADANIPORTS
-I
-II
-III
ALKEM
-I
-II
-III
AMBUJACEM
-I
-II
-III
APOLLOHOSP
-I
-II
-III
APOLLOTYRE
-I
-II
-III
ASHOKLEY
-I
-II
-III
ASIANPAINT
-I
-II
-III
ASTRAL
-I
-II
-III
ATUL
-I
-II
-III
AUBANK
-I
-II
-III
AUROPHARMA
-I
-II
-III
AXISBANK
-I
-II
-III
BAJAJFINSV
-I
-II
-III
BAJAJ_AUTO
-I
-II
-III
BAJFINANCE
-I
-II
-III
BALKRISIND
-I
-II
-III
BALRAMCHIN
-I
-II
-III
BANDHANBNK
-I
-II
-III
BANKBARODA
-I
-II
-III
BATAINDIA
-I
-II
-III
BEL
-I
-II
-III
BERGEPAINT
-I
-II
-III
BHARATFORG
-I
-II
-III
BHARTIARTL
-I
-II
-III
BHEL
-I
-II
-III
BIOCON
-I
-II
-III
BOSCHLTD
-I
-II
-III
BPCL
-I
-II
-III
BRITANNIA
-I
-II
-III
BSOFT
-I
-II
-III
CANBK
-I
-II
-III
CANFINHOME
-I
-II
-III
CHAMBLFERT
-I
-II
-III
CHOLAFIN
-I
-II
-III
CIPLA
-I
-II
-III
COALINDIA
-I
-II
-III
COFORGE
-I
-II
-III
COLPAL
-I
-II
-III
CONCOR
-I
-II
-III
COROMANDEL
-I
-II
-III
CROMPTON
-I
-II
-III
CU

In [14]:
print(numberOfRowsRaw)
print(numberOfRowsFinal)

531929
531929
